#  Tone Classification Task

## Data Preparation

In [1]:
require 'loader'
MAX_LEN = 256
-- Loader:new(obj, max_len)
ldr = Loader:new(nil, MAX_LEN)

In [2]:
function make_dataset(name)
    local dataset = {data = ldr.data[name], label = ldr.label[name]}
    function dataset:size() 
        return self.data:size(1) 
    end
    for i = 1, dataset:size() do
        dataset[i] = {dataset.data[i], dataset.label[i]}
    end
    for i = 1, 2 do
--         we abandon engy...
        if i == 2 then
            dataset.data[{{}, {}, i}]:fill(0)
            break
        end
        local std = dataset.data[{{}, {}, i}]:std()
        dataset.data[{{}, {}, i}]:div(std)
    end
    return dataset
end

In [3]:
trainset = make_dataset('train')
testset = make_dataset('test')
newtestset = make_dataset('test_new')

In [4]:
Plot = require 'itorch.Plot'
local x1 = torch.range(1,MAX_LEN)
local sampleN = 60
local y1 = newtestset.data[{sampleN,{},1}]
local y2 = newtestset.data[{sampleN,{},2}]
plot = Plot():line(x1, y1,'red','f0'):legend(true):title('Feature Observation of a Tone '..newtestset.label[sampleN]):draw()
plot:line(x1, y2,'blue','engy'):redraw()

In [5]:
require 'nn'
local inp = 2;  -- dimensionality of one sequence element 
local outp = 5; -- number of derived features for one sequence element
local conv_kw = 4;   -- number of sequence element kernel operates on per step
local conv_dw = 4;   -- step dw and go on to the next sequence element

local pool_kw = 4;   -- number of sequence element pooled on per step
local pool_dw = 4;   -- step dw and go on to the next sequence element

local nOutputFrame = (MAX_LEN - conv_kw) / conv_dw + 1
local nOutputFrame = (nOutputFrame - pool_kw) / pool_dw + 1

net = nn.Sequential()
net:add(nn.TemporalConvolution(inp, outp, conv_kw, conv_dw))
net:add(nn.ReLU())
net:add(nn.TemporalMaxPooling(pool_kw, pool_dw))
net:add(nn.View(outp*nOutputFrame))
net:add(nn.Linear(outp*nOutputFrame, 64))
net:add(nn.ReLU())
net:add(nn.Linear(64, 4))
net:add(nn.LogSoftMax())

In [6]:
print (net:__tostring())

nn.Sequential {
  [input -> (1) -> (2) -> (3) -> (4) -> (5) -> (6) -> (7) -> (8) -> output]
  (1): nn.TemporalConvolution
  (2): nn.ReLU
  (3): nn.TemporalMaxPooling
  (4): nn.View(80)
  (5): nn.Linear(80 -> 64)
  (6): nn.ReLU
  (7): nn.Linear(64 -> 4)
  (8): nn.LogSoftMax
}	


In [7]:
criterion = nn.ClassNLLCriterion();  
trainer = nn.StochasticGradient(net, criterion)  
trainer.learningRate = 0.001
trainer.maxIteration = 200
trainer.verbose = False

In [8]:
function cal_acc(dataset)
    local acc = 0.0
    for i = 1, dataset:size() do
        _, output = torch.max(net:forward(dataset[i][1]), 1)
        if output[1] == dataset[i][2] then
            acc = acc + 1.0
        end
    end
    acc = acc / dataset:size()
    return acc
end

In [9]:
recorder = {loss = {}, trainacc = {}, valacc = {}}
trainer.hookIteration = function(obj, iteration, currentError) 
    table.insert(recorder.loss, currentError)
    table.insert(recorder.trainacc, cal_acc(trainset))
    table.insert(recorder.valacc, cal_acc(testset))
    if (iteration % 10 == 0) then 
        print('Iteration '..iteration..': loss('..currentError..') train_acc('
            ..cal_acc(trainset)..') test_acc('..cal_acc(testset)..').') 
    end
end

In [10]:
trainer:train(trainset)

# StochasticGradient: training	


Iteration 10: loss(0.52528816998077) train_acc(0.8025) test_acc(0.85).	


Iteration 20: loss(0.33721775707839) train_acc(0.91) test_acc(0.9).	


Iteration 30: loss(0.27129143864792) train_acc(0.94) test_acc(0.925).	


Iteration 40: loss(0.24045226222827) train_acc(0.95) test_acc(0.975).	


...

Iteration 160: loss(0.11777277269579) train_acc(0.98) test_acc(0.975).	


Iteration 170: loss(0.11454427258382) train_acc(0.98) test_acc(0.975).	


Iteration 180: loss(0.11132242277332) train_acc(0.98) test_acc(0.975).	


Iteration 190: loss(0.10872066682485) train_acc(0.98) test_acc(0.975).	


Iteration 200: loss(0.10557050633437) train_acc(0.98) test_acc(0.975).	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 0.10557050633437	



In [11]:
local x = torch.range(1, #recorder.loss)
plt = Plot():line(x, recorder.loss,'red','loss'):legend(true):title('Train Loss'):draw()
plt = Plot():line(x, recorder.trainacc,'blue','train_acc'):legend(true):title('Accuracy'):draw()
plt:line(x, recorder.valacc,'green','val_acc'):redraw()

In [12]:
print ('accuracy on train set '..cal_acc(trainset))
print ('accuracy on test set '..cal_acc(testset))
print ('accuracy on new test set '..cal_acc(newtestset))

accuracy on train set 0.98	


accuracy on test set 0.975	


accuracy on new test set 0.76315789473684	
